# Homework 3
This homework has been assigned in date 8th of November 2024.
We have been required to complete a series of tasks on a *basket-item* kind dataset:
1. Apply the **fp-growth** algorithm to to detaset, present the most intersting results. 
2. Make a comparison between **apriori** and **fp-growth**.

We have been offered a dataset of transactions owned by a third party, as I have no rights over it, I assume we are not allowed to share, please reach out our professors of *Introduction to Data Mining* at [this link](https://web.dmi.unict.it/courses/l-31/course-units/?seuid=8EAB2D3A-4281-40F4-83A0-C6B007577BA2) if interested.

## Initialization
Once again, we are going to use the *CacheManager* from the previous homework, and we are going to initialize our script environment.


In [39]:
import os
import pandas as pd

from src.cache import CacheManager

dataset_path = os.getenv('TASK_DATASET_FILE_PATH')
cache = CacheManager().get_cache()


### Using a different dataset 
Due to the memory limitations of my system, for this time categories at level 4 are going to be used for our market-basket analysis:

In [40]:

trx_df = (
    cache.transactions
        .join(cache.items, on='cod_prod')
        ['liv4']
        .rename('cat_id')
        .to_frame()
)
trx_df


,cat_id
scontrino_id,
52597232,1180203
52597232,5040402
52597232,3080203
52597232,1150601
52597232,1150601
...,...
62224004,3060409
62228225,1050101
62228225,1050101


## Task 1: Using FP-Growth
Before using *FP-Growth*, it is mandatory to create a cross table of our transaction, in this case *to_sparse* will do just that:

In [41]:
from src.utils import to_sparse
sparse_trx_df = to_sparse(trx_df)
sparse_trx_df



,1010101,1010102,1010103,1010104,1010105,1010106,1010201,1010202,1010301,1010302,...,57011403,57020201,57021203,57021302,57021303,57021401,91010101,92010101,98010101,99999999
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153934,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
153935,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
153936,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
153937,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


We define a minimum support for our algorithm:

In [42]:
min_support = 0.001

A series of *bridge classes* around mlxtend's functions have been created, of such *FpGrowthAlgorithm* is the one that will resolve to mlxtend's FP-Growth.

In [43]:
from src.frequent_patterns import FpGrowthAlgorithm

fpgrowth = FpGrowthAlgorithm(sparse_trx_df).run(min_support = min_support)


The result of *FpGrowthAlgorithm* and the family of *FrequentPatternAlgorithm* is a *FrequentItemsets* object, which easily provide a method to extract rules:

In [44]:
rules_df = fpgrowth.extract_rules(min_threshold=.7).sort_values('confidence', ascending=False)
rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3886,(8013502),(8013501),0.001728,0.002800,0.001728,1.0,357.167053,0.001723,inf,0.998926
3071,"(8010304, 24010102, 3590102)",(8010203),0.001189,0.045895,0.001189,1.0,21.788960,0.001134,inf,0.955241
3248,"(8010304, 3060406, 3010102)",(8010203),0.001338,0.045895,0.001338,1.0,21.788960,0.001277,inf,0.955384
3829,"(3070204, 3010102)",(3091001),0.001189,0.010985,0.001189,1.0,91.034299,0.001176,inf,0.990192
3129,"(8010304, 3080802)",(8010203),0.001475,0.045895,0.001475,1.0,21.788960,0.001407,inf,0.955514
...,...,...,...,...,...,...,...,...,...,...
2705,"(9040201, 1210402, 9010201)",(9010101),0.001689,0.168801,0.001182,0.7,4.146904,0.000897,2.770665,0.760140
1876,"(3060401, 24010102, 3011515, 3010102)",(3060402),0.001689,0.210505,0.001182,0.7,3.325329,0.000827,2.631648,0.700461
2296,"(1150201, 3011515, 3110108, 9010201)",(9010101),0.002404,0.168801,0.001682,0.7,4.146904,0.001277,2.770665,0.760685
1902,"(3060401, 3060405, 1060501)",(3060402),0.001689,0.210505,0.001182,0.7,3.325329,0.000827,2.631648,0.700461


We can resolve descriptions from the previous results, as it easier to us to understand data in a human readable form:

In [45]:
from src.utils import resolve_descriptions
rules_df['descriptions'] = resolve_descriptions(rules_df, cache.categories)
rules_df


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,descriptions
3886,(8013502),(8013501),0.001728,0.002800,0.001728,1.0,357.167053,0.001723,inf,0.998926,ESTERI => NAZIONALI
3071,"(8010304, 24010102, 3590102)",(8010203),0.001189,0.045895,0.001189,1.0,21.788960,0.001134,inf,0.955241,"CRUDITE', PANETTERIA PANE, PREPARATI GASTRON.(..."
3248,"(8010304, 3060406, 3010102)",(8010203),0.001338,0.045895,0.001338,1.0,21.788960,0.001277,inf,0.955384,"CRUDITE', BRESAOLA, LATTE VACCINO => INSALATE ..."
3829,"(3070204, 3010102)",(3091001),0.001189,0.010985,0.001189,1.0,91.034299,0.001176,inf,0.990192,"ALTRI, LATTE VACCINO => SECONDI PIATTI RICOMPOSTI"
3129,"(8010304, 3080802)",(8010203),0.001475,0.045895,0.001475,1.0,21.788960,0.001407,inf,0.955514,"CRUDITE', PANE COMUNE => INSALATE TENERE UNITIPO"
...,...,...,...,...,...,...,...,...,...,...,...
2705,"(9040201, 1210402, 9010201)",(9010101),0.001689,0.168801,0.001182,0.7,4.146904,0.000897,2.770665,0.760140,"AVICUNICOLO, BOTTIGLIA/VASO, AVICUNICOLO => BO..."
1876,"(3060401, 24010102, 3011515, 3010102)",(3060402),0.001689,0.210505,0.001182,0.7,3.325329,0.000827,2.631648,0.700461,"PROSCIUTTO CRUDO, PANETTERIA PANE, GRANA E SIM..."
2296,"(1150201, 3011515, 3110108, 9010201)",(9010101),0.002404,0.168801,0.001682,0.7,4.146904,0.001277,2.770665,0.760685,"NORM.ASC.CORTA, GRANA E SIMILI, ALLEVATE A TER..."
1902,"(3060401, 3060405, 1060501)",(3060402),0.001689,0.210505,0.001182,0.7,3.325329,0.000827,2.631648,0.700461,"PROSCIUTTO CRUDO, MORTADELLA, LATTA => PROSCIU..."


## Task 2: Comparison with Apriori


Comparison with Apriori is straightforward:  
> *Notes*: my device has not enough memory to run Apriori without low_memory's flag of mlxtend, a standard version, **AprioriAlgorithm**, is available for any curiosity.

In [46]:
from src.frequent_patterns import AprioriLowMemAlgorithm

apriori = AprioriLowMemAlgorithm(sparse_trx_df).run(min_support)
apriori.df

,support,itemsets
0,0.005652,(1010101)
1,0.012492,(1010102)
2,0.012765,(1010103)
3,0.041302,(1010104)
4,0.008205,(1010105)
...,...,...
78500,0.001007,"(3060402, 9010101, 9010102, 21010103, 3011512,..."
78501,0.001221,"(3060402, 9010101, 21010103, 3011512, 9010201,..."
78502,0.001195,"(3060402, 9010101, 9010102, 21010103, 3011512,..."
78503,0.001000,"(3060402, 9010101, 21010103, 9010201, 3011515,..."


We can compare Apriori's produced itemsets with *FrequentItemsets's compare* method, and assert it is equal, at least, in our case.

In [47]:
compare_df = apriori.compare(fpgrowth).df
assert (compare_df['support_x'] == compare_df['support_y']).all()

So far, it is not even worth the trouble to use Apriori, if not for educational purposes.